In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [5]:
import numpy as np
from tqdm import tqdm
from pathlib import Path
import openslide
import pandas as pd
import pickle

In [6]:
from fastai.callbacks.csv_logger import CSVLogger

In [7]:
from object_detection_fastai.helper.object_detection_helper import *
from object_detection_fastai.helper.wsi_loader import *
from object_detection_fastai.loss.RetinaNetFocalLoss import RetinaNetFocalLoss
from object_detection_fastai.models.RetinaNet import RetinaNet
from object_detection_fastai.callbacks.callbacks import BBLossMetrics, BBMetrics, PascalVOCMetric, PascalVOCMetricByDistance

In [9]:
slides_train = list(set(["BAL_Cat_Turnbull blue_13.svs", 
                         "BAL_Cat_Turnbull blue_6.svs", 
                         "BAL_Cat_Turnbull blue_12.svs", 
                         "BAL_Cat_Turnbull blue_14.svs", 
                         "BAL_Cat_Turnbull blue_2.svs", ]))

slides_val = list(set(["BAL_Cat_Turnbull blue_1.svs", "BAL_Cat_Turnbull blue_10.svs"]))

In [10]:
grade_list = [0, 1]
annotations_path = Path("../../Statistics/EIPH_Annotations.pkl")
annotations = pd.read_pickle(annotations_path)
annotations["image_name"] = [name.replace("tiff","svs") for name in annotations["image_name"]]
annotations = annotations[annotations["grade"].isin(grade_list)]
annotations_train = annotations[annotations["image_name"].isin(slides_train)]
annotations_val = annotations[annotations["image_name"].isin(slides_val)]
annotations_train.head()

,id,image_id,image_set,species,image_name,image_type,grade,vector,unique_identifier,user_id,deleted,last_editor,data_set_name,version
100398,1421918,3518,245,equine,07_EIPH_574882 left lung Turnbull blue.svs,TurnbullBlue,0,"{'x1': 19619, 'x2': 19747, 'y1': 3009, 'y2': 3...",87e3870c-2316-4689-beaf-d2d6ddf22c28,1,True,1,SREP,SREP
100399,1421919,3518,245,equine,07_EIPH_574882 left lung Turnbull blue.svs,TurnbullBlue,0,"{'x1': 20116, 'x2': 20202, 'y1': 3309, 'y2': 3...",ff289b7a-2fce-4d37-9feb-74b592d842d9,1,True,1,SREP,SREP
100400,1421922,3518,245,equine,07_EIPH_574882 left lung Turnbull blue.svs,TurnbullBlue,0,"{'x1': 14193, 'x2': 14279, 'y1': 3885, 'y2': 3...",6ffcd9c4-1e54-4b9b-b593-4f2c9e85704d,1,True,1,SREP,SREP
100401,1421923,3518,245,equine,07_EIPH_574882 left lung Turnbull blue.svs,TurnbullBlue,0,"{'x1': 14271, 'x2': 14347, 'y1': 3767, 'y2': 3...",1c3cb4f8-fad5-4b13-8859-5b83f31e450a,1,True,1,SREP,SREP
100402,1421924,3518,245,equine,07_EIPH_574882 left lung Turnbull blue.svs,TurnbullBlue,0,"{'x1': 14782, 'x2': 14860, 'y1': 3853, 'y2': 3...",682b6be5-664a-4bd0-ae5a-12a7963a2dd4,1,True,1,SREP,SREP


In [12]:
slides_path = Path("../../Slides")
files = {slide.name: slide for slide in slides_path.rglob("*.svs")  if slide.name in slides_train + slides_val}
files

{'01_EIPH_563479 Berliner Blau.svs': WindowsPath('D:/Datasets/EIPH WSI/Pferd/Berliner Blau/01_EIPH_563479 Berliner Blau.svs'),
 '02_EIPH_574162 berliner blau.svs': WindowsPath('D:/Datasets/EIPH WSI/Pferd/Berliner Blau/02_EIPH_574162 berliner blau.svs'),
 '03_EIPH_566933 R Berliner Blau neu.svs': WindowsPath('D:/Datasets/EIPH WSI/Pferd/Berliner Blau/03_EIPH_566933 R Berliner Blau neu.svs'),
 '04_EIPH_567017 Berliner Blau.svs': WindowsPath('D:/Datasets/EIPH WSI/Pferd/Berliner Blau/04_EIPH_567017 Berliner Blau.svs'),
 '05_EIPH_569923 Berliner Blau.svs': WindowsPath('D:/Datasets/EIPH WSI/Pferd/Berliner Blau/05_EIPH_569923 Berliner Blau.svs'),
 '11_EIPH_575697 Berliner Blau.svs': WindowsPath('D:/Datasets/EIPH WSI/Pferd/Berliner Blau/11_EIPH_575697 Berliner Blau.svs'),
 '20_EIPH_576150 berliner blau.svs': WindowsPath('D:/Datasets/EIPH WSI/Pferd/Berliner Blau/20_EIPH_576150 berliner blau.svs'),
 '26_EIPH_566482 L Berliner Blau.svs': WindowsPath('D:/Datasets/EIPH WSI/Pferd/Berliner Blau/26_EIP

In [15]:
tfms = get_transforms(do_flip=True,
                      flip_vert=True,
                      #max_rotate=90,
                      max_lighting=0.0,
                      max_zoom=1.,
                      max_warp=0.0,
                      p_affine=0.5,
                      p_lighting=0.0,
                      #xtra_tfms=xtra_tfms,
                     )
tfms

([RandTransform(tfm=TfmCrop (crop_pad), kwargs={'row_pct': (0, 1), 'col_pct': (0, 1), 'padding_mode': 'reflection'}, p=1.0, resolved={}, do_run=True, is_random=True, use_on_y=True),
  RandTransform(tfm=TfmAffine (dihedral_affine), kwargs={}, p=1.0, resolved={}, do_run=True, is_random=True, use_on_y=True),
  RandTransform(tfm=TfmAffine (rotate), kwargs={'degrees': (-10.0, 10.0)}, p=0.5, resolved={}, do_run=True, is_random=True, use_on_y=True)],
 [RandTransform(tfm=TfmCrop (crop_pad), kwargs={}, p=1.0, resolved={}, do_run=True, is_random=True, use_on_y=True)])

In [13]:
size = 1024 
level = 0
bs = 16
train_images = 2500
val_images = 1500

In [3]:
def get_y_func(x):
    return x.y

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
anchors = create_anchors(sizes=[(32,32)], ratios=[1], scales=[0.6, 0.7, 0.9, 1.25, 1.5])

In [14]:
for i in range(len(slides_train)):

    train_files = []
    val_files = []
    
    experiment_name = "CatVsCat-Ablation_{}".format(i)

    for image_name in srep_slides_train[:i+1]:

        annotations = annotations_train[annotations_train["image_name"] == image_name]
        annotations = annotations[annotations["deleted"] == False]

        slide_path = files[image_name]
        labels =  list(annotations["grade"])
        bboxes = [[vector["x1"], vector["y1"], vector["x2"], vector["y2"]] for vector in annotations["vector"]]

        train_files.append(SlideContainer(slide_path, y=[bboxes, labels],  level=level, width=size, height=size))

    for image_name in annotations_val["image_name"].unique():

        annotations = annotations_val[annotations_val["image_name"] == image_name]
        annotations = annotations[annotations["deleted"] == False]

        slide_path = files[image_name]
        labels =  list(annotations["grade"])
        bboxes = [[vector["x1"], vector["y1"], vector["x2"], vector["y2"]] for vector in annotations["vector"]]

        val_files.append(SlideContainer(slide_path, y=[bboxes, labels],  level=level, width=size, height=size))
    
    train_files = list(np.random.choice(train_files, train_images))
    valid_files = list(np.random.choice(val_files, val_images))
    
    train =  ObjectItemListSlide(train_files, path=slides_path)
    valid = ObjectItemListSlide(valid_files, path=slides_path)
    item_list = ItemLists(slides_path, train, valid)
    lls = item_list.label_from_func(get_y_func, label_cls=SlideObjectCategoryList) #
    lls = lls.transform(tfms, tfm_y=True, size=size)
    data = lls.databunch(bs=bs, collate_fn=bb_pad_collate, num_workers=0).normalize()
    
    crit = RetinaNetFocalLoss(anchors)
    encoder = create_body(models.resnet18, True, -2)
    model = RetinaNet(encoder, n_classes=data.train_ds.c, n_anchors=5, sizes=[32], chs=128, final_bias=-4., n_conv=3)
    
    voc = PascalVOCMetricByDistance(anchors, size, [str(i) for i in data.train_ds.y.classes[1:]], radius=25)
    learn = Learner(data, model, loss_func=crit, callback_fns=[BBMetrics, partial(CSVLogger, append=False, filename=experiment_name)], #BBMetrics, ShowGraph
                    metrics=[voc])

    learn.split([model.encoder[6], model.c5top5])
    learn.freeze_to(-2)
    
    
    learn.unfreeze()
    learn.fit_one_cycle(10, 1e-3)
    
    stats = {"anchors": anchors,
         "mean": to_np(data.stats[0]),
         "std": to_np(data.stats[1]),
         "size": size,
         "n_classes": 6,
         "n_anchors": 5,
         "sizes": [32],
         "chs": 128,
         "encoder": "RN-18",
         "n_conv": 3,
         "level": 0,
         "model": get_model(learn.model).state_dict()
        }

    torch.save(stats, "{}.p".format(experiment_name))

In [ ]:
slide_object_result(learn, anchors, detect_thresh=0.3, nms_thresh=0.2, image_count=20)